# Nimbus Backend Demo

This notebook demonstrates how to set up and use the Hamilton Nimbus backend with PyLabRobot's `LiquidHandler` interface.

The demo covers:
1. Creating a `NimbusBackend` instance
2. Creating a `NimbusDeck` (using defaults or from config files)
3. Creating a `LiquidHandler` with the backend and deck
4. Setting up the robot
5. Parking the pipette
6. Demonstrating door lock operations (if available)
7. Closing the connection


In [1]:
# Import necessary modules
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.liquid_handling.backends.hamilton.nimbus_backend import NimbusBackend
from pylabrobot.resources.hamilton.nimbus_decks import NimbusDeck


In [ ]:
# Create NimbusBackend instance
# Replace with your instrument's IP address
backend = NimbusBackend(
    host="192.168.100.100",  # Replace with your instrument's IP
    port=2000,
    read_timeout=30,
    write_timeout=30
)

# Create NimbusDeck using default values (layout 8 dimensions)
deck = NimbusDeck()

# Alternatively, you can load from config files:
# deck = NimbusDeck.from_files(
#     cfg_path="/path/to/Nimbus8.cfg",
#     dck_path="/path/to/Nimbus8.dck"
# )

print(f"Deck created: {deck.name}")
print(f"  Size: {deck.get_size_x()} x {deck.get_size_y()} x {deck.get_size_z()} mm")
print(f"  Rails: {deck.num_rails}")
deck.serialize()


Deck created: deck
  Size: 831.85 x 424.18 x 300.0 mm
  Rails: 30


{'name': 'deck',
 'type': 'NimbusDeck',
 'size_x': 831.85,
 'size_y': 424.18,
 'size_z': 300.0,
 'location': {'x': 0, 'y': 0, 'z': 0, 'type': 'Coordinate'},
 'rotation': {'x': 0, 'y': 0, 'z': 0, 'type': 'Rotation'},
 'category': 'deck',
 'barcode': None,
 'children': [],
 'parent_name': None,
 'num_rails': 30,
 'with_trash': False,
 'with_trash96': False,
 'hamilton_origin': {'x': -151.51, 'y': -363.83, 'z': 0.0},
 'y_min': -310.0,
 'y_max': 20.0,
 'z_max': 146.0,
 'rail_start_x': -125.7,
 'rail_width': 22.454,
 'rail_y': -360.487}

In [3]:
# Create LiquidHandler with backend and deck
lh = LiquidHandler(backend=backend, deck=deck)

print("LiquidHandler created successfully")

# Setup the robot
# This will:
# - Connect to the instrument via TCP
# - Discover instrument objects (Pipette, DoorLock, NimbusCore)
# - Lock the door if available
# - Pre-initialize the pipette
# - Query tip presence
# - Query channel configuration
# Optionally unlock door after pre-initialization:
await lh.setup(unlock_door=False)

print("\n" + "="*60)
print("SETUP COMPLETE")
print("="*60)
print(f"Setup finished: {backend.setup_finished}")
print(f"\nConnection Info:")
print(f"  Client ID: {backend._client_id}")
print(f"  Client Address: {backend.client_address}")
print(f"\nDiscovered Objects:")
print(f"  NimbusCore Root: {backend._nimbus_core_address}")
print(f"  Pipette: {backend._pipette_address}")
if backend._door_lock_address is not None:
    print(f"  DoorLock: {backend._door_lock_address}")
else:
    print(f"  DoorLock: Not available on this instrument")
print(f"\nInstrument Configuration:")
print(f"  Number of channels: {backend.num_channels}")


LiquidHandler created successfully

SETUP COMPLETE
Setup finished: True

Connection Info:
  Client ID: 7
  Client Address: 2:7:65535

Discovered Objects:
  NimbusCore Root: 1:1:48896
  Pipette: 1:1:257
  DoorLock: 1:1:268

Instrument Configuration:
  Number of channels: 4


In [4]:
# Park the pipette channels
await lh.backend.park()

print("Pipette parked successfully")

# Check if door lock is available and demonstrate operations
# Check door lock status
is_locked = await lh.backend.is_door_locked()
print(f"Door is currently: {'locked' if is_locked else 'unlocked'}")

# Unlock the door
await lh.backend.unlock_door()
print("Door unlocked")

# Check status again
is_locked = await lh.backend.is_door_locked()
print(f"Door is now: {'locked' if is_locked else 'unlocked'}")


Pipette parked successfully
Door is currently: locked
Door unlocked
Door is now: unlocked


In [5]:
# Stop and close connection
await lh.stop()

print("Connection closed successfully")


Connection closed successfully
